In [272]:
# Import necessary libraries
import torch
import numpy as np
import pandas as pd
import diffprivlib.models as dp
from sklearn.linear_model import LogisticRegression
from torch import nn, optim
from opacus import PrivacyEngine
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

In [273]:
# bias
# holisticai imports
import holisticai
from holisticai.bias.metrics import classification_bias_metrics
from holisticai.bias.mitigation.postprocessing import CalibratedEqualizedOdds, RejectOptionClassification

## Load Training and Testing Data

In [274]:
# Load the data
X_train = pd.read_csv('../../data/X_train.csv',index_col=0).drop(['id'],axis=1)
y_train = pd.read_csv('../../data/y_train.csv',index_col=0)
X_test = pd.read_csv('../../data/X_test.csv',index_col=0).drop(['id'],axis=1)
y_test = pd.read_csv('../../data/y_test.csv',index_col=0)

In [275]:
baseline__df = pd.read_csv("../../data/data_baseline.csv",index_col=0)
baseline__df = baseline__df.drop(['age_cat'],axis=1)

## Transform Data for the Model

In [276]:
# Check type of y_test
type(y_test)

pandas.core.frame.DataFrame

In [277]:
y_test.head()

,two_year_recid
3613,0
4233,0
904,0
5365,1
5705,0


In [278]:
group_a_train = X_train["African-American_race"] == 1
group_b_train = X_train["African-American_race"] == 0
data_train = [X_train, y_train, group_a_train, group_b_train]


group_a_test = X_test["African-American_race"] == 1
group_b_test = X_test["African-American_race"] == 0
data_test = [X_test, y_test, group_a_test, group_b_test]


Defining the functions to calcluate the perfomace metrics

In [279]:
# efficacy metrics from sklearn
from sklearn import metrics

# dictionnary of metrics
metrics_dict={
        "Accuracy": metrics.accuracy_score,
        "Balanced accuracy": metrics.balanced_accuracy_score,
        "Precision": metrics.precision_score,
        "Recall": metrics.recall_score,
        "F1-Score": metrics.f1_score}

# efficacy metrics dataframe helper tool
def metrics_dataframe(y_pred, y_true, metrics_dict=metrics_dict):
    metric_list = [[pf, fn(y_true, y_pred)] for pf, fn in metrics_dict.items()]
    return pd.DataFrame(metric_list, columns=["Metric", "Value"]).set_index("Metric")

In [280]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [281]:
# Check the shapes of the scaled data
X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape

((5748, 23), (1437, 23), (5748, 1), (1437, 1))

### 1. Baseline Logistic Regression Model

In [282]:
baseline_lr = LogisticRegression(solver="lbfgs", max_iter=1000)
baseline_lr.fit(X_train_scaled, y_train)

c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000)

In [283]:
y_pred = baseline_lr.predict(X_test_scaled)
y_proba = baseline_lr.predict_proba(X_test_scaled)
y_score = y_proba[:,1]
y_true = y_test

Model Performance

In [284]:
# Baseline efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.700765
Balanced accuracy,0.691618
Precision,0.694097
Recall,0.599691
F1-Score,0.643449


Fairness Performance

In [285]:
fairness_metrics = classification_bias_metrics(group_a_test, group_b_test, y_pred, y_true, metric_type='both')
fairness_metrics.iloc[6]

Value        0.157859
Reference    0.000000
Name: False Positive Rate Difference, dtype: float64

Difference is false positivity rate very high

### 2. Differentially Private Logistic Regression Model


epsilon (ε) represents the maximum acceptable difference in the model's output when the dataset is modified by adding or removing a single data point. 
Intuition: 
When epsilon is provided to a logistic regression model, it guides the model to learn a representation of the data that is less sensitive to individual changes.

By introducing epsilon, the logistic regression model limits the amount of information that can be inferred about any single individual. This is achieved by adding noise to the model's parameters or predictions, making it difficult for an adversary to discern the impact of any single data point.

This is equivalent to learning Fair Representation in terms of Fairness, whereas individual's association witht the outcome is obfuscated, and therefore we expect to see fairer model just by introducing the privacy budget

In [286]:
dp_clf = dp.LogisticRegression(random_state=0,epsilon=13)
dp_clf.fit(X_train_scaled, y_train)

c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\athar\anaconda3\lib\site-packages\diffprivlib\models\logistic_regression.py:239: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "


LogisticRegression(accountant=BudgetAccountant(spent_budget=[(1.0, 0), (4, 0), (8, 0), (50, 0), (15, 0), ...]),
                   data_norm=40.8668907569636, epsilon=13, random_state=0)

In [287]:
y_pred_dp = dp_clf.predict(X_test_scaled)
y_proba_dp = dp_clf.predict_proba(X_test_scaled)
y_score_dp = y_proba_dp[:,1]

In [288]:
# Baseline efficacy
metrics_dataframe(y_pred_dp, y_true)

,Value
Metric,
Accuracy,0.688935
Balanced accuracy,0.676941
Precision,0.692308
Recall,0.556414
F1-Score,0.616967


In [289]:
fairness_metrics_dp = classification_bias_metrics(group_a_test, group_b_test, y_pred_dp, y_true, metric_type='both')
fairness_metrics_dp.iloc[6]

Value        0.085616
Reference    0.000000
Name: False Positive Rate Difference, dtype: float64

In [290]:
print("Privacy Measure : ", dp_clf.epsilon)
print("Fairness measure: ", fairness_metrics_dp.iloc[6][0])

Privacy Measure :  13
Fairness measure:  0.08561643835616439


<h3> Implementing Fairness

1. Caliberated Equalized Odds

In [291]:
# initialize object
ceo = CalibratedEqualizedOdds(cost_constraint="fpr")

In [292]:
# predict train set
y_pred_train_dp = dp_clf.predict(X_train_scaled)
y_proba_train_dp = dp_clf.predict_proba(X_train_scaled)

In [293]:
# fit it
ceo.fit(y_train, y_proba_train_dp, group_a_train, group_b_train)

In [310]:
ceo.transform?

Signature: ceo.transform(*args, **kargs)
Docstring:
Apply transform function to predictions and likelihoods

Description
----------
Use a fitted probability to change the output label and invert the likelihood

Parameters
----------
y_pred : array-like
    Predicted vector (nb_examlpes,)
y_proba : matrix-like
    Predicted probability matrix (num_examples, num_classes). The probability
    estimates must sum to 1 across the possible classes and each matrix value
    must be in the interval [0,1].
group_a : array-like
    Group membership vector (binary)
group_b : array-like
    Group membership vector (binary)
threshold : float
    float value to discriminate between 0 and 1

Returns
-------
dictionnary with new predictions
File:      c:\users\athar\anaconda3\lib\site-packages\holisticai\utils\transformers\_transformer_base.py
Type:      function


In [294]:
# transform it
d = ceo.transform(y_test, y_proba_dp, group_a_test, group_b_test, 0.8)
# new predictions
y_pred_ceo = d['y_pred']

In [295]:
# efficacy
metrics_dataframe(y_pred_ceo, y_test)

,Value
Metric,
Accuracy,0.589422
Balanced accuracy,0.547686
Precision,0.761468
Recall,0.128284
F1-Score,0.219577


In [296]:
fairness_metrics_ceo = classification_bias_metrics(group_a_test, group_b_test, y_pred_ceo, y_test, metric_type='both')
print("Fairness metric : ", fairness_metrics_ceo.iloc[6][0])

Fairness metric :  0.0533675799086758


In [ ]:
#Creating the dataframe for the different values of discrimination threshold


#Looping through the diffent values 
for i in range(0,1,0.1):
	# transform it
	d = ceo.transform(y_test, y_proba_dp, group_a_test, group_b_test, 0.8)
	# new predictions
	y_pred_ceo = d['y_pred']
	


Fairness got worse

<h3> Reject Option Classification

In [297]:
#changing the lables since ROC considers 1 as fav label, and optimizes for it
y_train_flipped = y_train.copy()
y_train_flipped['two_year_recid'] = np.where(y_train_flipped['two_year_recid'] == 1,0,1)

y_test_flipped = y_test.copy()
y_test_flipped['two_year_recid'] = np.where(y_test_flipped['two_year_recid'] == 1,0,1)

fitting new lr model with swapped labels

In [298]:
# dp_clf2 = dp.LogisticRegression(random_state=0,epsilon=13)
# dp_clf2.fit(X_train_scaled, y_train_flipped)

# # predict train set
# y_pred_train_dp_flipped = dp_clf2.predict(X_train_scaled)
# y_proba_train_dp_flipped = dp_clf2.predict_proba(X_train_scaled)

# # predict test set
# y_pred_test_dp_flipped = dp_clf2.predict(X_test_scaled)
# y_proba_test_dp_flipped = dp_clf2.predict_proba(X_test_scaled)
# y_score_test_dp_flipped = y_proba_test_dp_flipped[:,1]

In [299]:
# #hold
# y_proba_train_dp_flipped = y_proba_train_dp.copy()
# y_proba_train_dp_flipped = y_proba_train_dp_flipped[:,[1,0]]

# y_proba_dp_flipped = y_proba_dp.copy()
# y_proba_dp_flipped = y_proba_dp_flipped[:,[1,0]]

In [300]:
# initialize
roc = RejectOptionClassification(metric_name="Statistical parity difference")
# fit it
roc.fit(y_train, y_proba_train_dp, group_a_train, group_b_train)

In [301]:
d = roc.transform(y_test, y_proba_dp, group_a_test, group_b_test)
d

{'y_pred': array([False, False, False, ..., False, False, False]),
 'y_score': array([0.15374175, 0.19360252, 0.51978189, ..., 0.47529827, 0.39096627,
        0.49288194])}

In [302]:
# new predictions
y_pred_roc = d['y_pred']
y_pred_roc

array([False, False, False, ..., False, False, False])

In [303]:
# efficacy
metrics_dataframe(y_pred_roc, y_test)

,Value
Metric,
Accuracy,0.599861
Balanced accuracy,0.557740
Precision,0.852941
Recall,0.134467
F1-Score,0.232310


In [304]:
# bias metrics
classification_bias_metrics(group_a_test, group_b_test, y_pred_roc, y_test, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,0.050629,0
Disparate Impact,2.118579,1
Four Fifths Rule,0.472015,1
Cohen D,0.198121,0
2SD Rule,3.736422,0
Equality of Opportunity Difference,0.058358,0
False Positive Rate Difference,0.016994,0
Average Odds Difference,0.037676,0
Accuracy Difference,-0.099952,0


In [305]:
X_test_cp = X_test.copy()

X_test_cp['y_pred_lrb'] = y_pred
X_test_cp['y_score_lrb'] = y_score

X_test_cp['y_pred_dp'] = y_pred_dp
X_test_cp['y_score_dp'] = y_score_dp

X_test_cp['y_pred_roc'] = d['y_pred']
X_test_cp['y_score_roc'] = d['y_score']

In [306]:
X_test_cp.columns

Index(['sex', 'age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count',
       'priors_count', 'c_charge_desc', 'time_in_custody_in_days',
       'time_in_jail_in_days', 'African-American_race', 'Asian_race',
       'Caucasian_race', 'Hispanic_race', 'Native American_race', 'Other_race',
       'Felony_c_charge_degree', 'Misdemeanor_c_charge_degree',
       'High_score_text', 'Low_score_text', 'Medium_score_text',
       'High_v_score_text', 'Low_v_score_text', 'Medium_v_score_text',
       'y_pred_lrb', 'y_score_lrb', 'y_pred_dp', 'y_score_dp', 'y_pred_roc',
       'y_score_roc'],
      dtype='object')

In [307]:
X_test_cp[['African-American_race','y_pred_lrb','y_score_lrb']].groupby(['African-American_race']).aggregate(['min','max'])

y_pred_lrb     y_score_lrb          
                             min max         min       max
African-American_race                                     
0                              0   1    0.025451  0.986105
1                              0   1    0.026688  0.997501

In [308]:
X_test_cp[['African-American_race','y_pred_roc','y_score_roc']].groupby(['African-American_race']).aggregate(['min','max'])

y_pred_roc       y_score_roc          
                             min   max         min       max
African-American_race                                       
0                          False  True    0.022286  0.997275
1                          False  True    0.018326  0.990368

In [309]:
d_df = pd.DataFrame(d)
d_df.groupby('y_pred').aggregate(['min','max'])

y_score          
             min       max
y_pred                    
False   0.018326  0.860765
True    0.861981  0.997275